# Mobility data visualiser for pymobility-generated Gauss-Markov location iterators
---
To successfully run this make sure you are in an environment that supports scikit-mobility.

# Imports
---
Cell also includes some baseline settings for static libraries

In [49]:
import numpy as np
import pandas as pd
import skmob
from tqdm import tqdm

tqdm.pandas()

# import seaborn as sns
#
# sns.set(style="white", color_codes=True)
# sns.set_context(rc={"font.family": 'sans', "font:size": 24, "axes.titlesize": 24, "axes.lavelsize": 24})

import warnings

warnings.filterwarnings("ignore")

from pymobility.models.mobility import gauss_markov

# Set metaparameters
---

## Parameters for the Gauss-Markov generator

In [50]:
POP_SIZE = 30
TOTAL_AREA_DIMS = (10000, 10000)
ALPHA = .5
VELOCITY_MEAN = 7.
VARIANCE = 7.
SIM_DURATION = 1440

# Define minutely generator
---

In [51]:
class MinutelyMovement:
    def __init__(self, movement_iter):
        self._move = movement_iter

    def __next__(self):
        for i in range(60):
            self_next = next(self._move)

        return self_next

# Generator creation
---

In [52]:
mobility_generator = gauss_markov(nr_nodes=POP_SIZE,
                                  dimensions=TOTAL_AREA_DIMS,
                                  velocity_mean=VELOCITY_MEAN,
                                  alpha=ALPHA,
                                  variance=VARIANCE)

minutely_gen = MinutelyMovement(movement_iter=mobility_generator)

# Randomize initial locations by running the iterator 500 times

In [53]:
for i in range(500):
    content = next(minutely_gen)

    big_mob_df = pd.DataFrame(content, columns=['x', 'y'])
    big_mob_df.insert(0, "uid", range(len(content)))
    big_mob_df.insert(0, "time_step", np.zeros(len(content), dtype=int))

# Iterate SIM_DURATION times and add to the dataframe

In [54]:
for i in range(SIM_DURATION):
    mob_list = next(minutely_gen)

    aux_df = pd.DataFrame(mob_list, columns=['x', 'y'], index=range((i + 1) * len(mob_list), (i + 2) * len(mob_list)))
    aux_df.insert(0, "uid", range(len(mob_list)))
    aux_df.insert(0, "time_step", np.zeros(len(mob_list), dtype=int) + i)

    frames = [big_mob_df, aux_df]
    big_mob_df = pd.concat(frames)

# Center around mean on both axes

In [55]:
big_mob_df['x'] -= big_mob_df['x'].mean()
big_mob_df['y'] -= big_mob_df['y'].mean()

# Translate metres to latitude/longitude

In [56]:
big_mob_df[['x', 'y']] *= 9.00900901e-6

# Center around Cluj-Napoca's city centre coordinates

In [57]:
big_mob_df['x'] += 46.7712
big_mob_df['y'] += 23.6236

# Produce skmob dataframe

In [58]:
skmob_cluj = skmob.TrajDataFrame(data=big_mob_df,
                                 latitude='x',
                                 longitude='y',
                                 datetime='time_step',
                                 user_id='uid')

In [59]:
skmob_cluj

,datetime,uid,lat,lng
0,1970-01-01 00:00:00.000000000,0,46.804410,23.637027
1,1970-01-01 00:00:00.000000000,1,46.808604,23.609503
2,1970-01-01 00:00:00.000000000,2,46.811882,23.671577
3,1970-01-01 00:00:00.000000000,3,46.741664,23.661376
4,1970-01-01 00:00:00.000000000,4,46.770838,23.634172
...,...,...,...,...
43225,1970-01-01 00:00:00.000001439,25,46.806235,23.609903
43226,1970-01-01 00:00:00.000001439,26,46.809663,23.655055
43227,1970-01-01 00:00:00.000001439,27,46.765507,23.591767
43228,1970-01-01 00:00:00.000001439,28,46.758373,23.631733


# Plot trajectories

In [60]:
skmob_cluj.plot_trajectory()